<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.851 - Tipología y ciclo de vida de los datos aula 1 · Práctica 1</p>
<p style="margin: 0; text-align:right;">2018 · Máster universitario en Ciencia de datos (Data science)</p>
<p style="margin: 0; text-align:right;">Prof. Colaboradora: <b>Laia Subirats Maté</b></p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Alumno: <b>Fernando Antonio Barbeiro Campos</b> - <a href="">fbarbeiro@uoc.edu</a></p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>



### Dataset
Historial de eventos del UFC (*Ultimate Fighting Championship*)
### Descripción
Un dataset completo con todos los eventos deportivos de MMA (*mixed martial arts*) más famoso del mundo, los deportistas, resultados, entre otros.

### Imagen identificativa

<img src="ufc-results.jpg" alt="UFC results" style="width: 700px;"/>
<center>Figura 1: Resultados y card de peleas de un evento de UFC.</center> 

### Contexto
El conjunto de datos se trata de todos los eventos deportivos de MMA realizados desde la creación del UFC (primer evento en el **12 de Noviembre de 1993**) hasta el último que ocurrió en el **27 de Octubre de 2018**, o sea, **454 eventos**, bien como el resultado de cada una de las peleas que ocurrieron en cada evento - precisamente **4869 luchas**.

### Contenido
El dataset tiene un contenido bastante sencillo que consiste en:
* [String] *event_name:* Nombre del evento
* [String] *weight_class:* Categoría de peso de la pelea
* [String] *fighter_1:* Nombre del competidor 1
* [String] *action:* Acción que ejecutó el competidor 1 sobre el oponente / resultado
* [String] *fighter_2:* Nombre del oponente
* [Integer] *round:* *Round* en que acabó la lucha
* [String] *time:* Tiempo en el *round* que acabó la lucha
* [String] *method:* Como ha sido el método utilizado para ganar

El intervalo de tiempo fuera mencionado arriba, vale resaltar que en cada evento hay una media de 10.7 luchas. La estrategia utilizada para recoger los datos es el resultado de iteraciones en listas:
* Inicialmente, pillamos la [lista de todos los eventos del UFC](https://en.wikipedia.org/wiki/List_of_UFC_events)
* Hacemos un `for` loop en la lista completa y extraemos (con uso de BeautifulSoup) las URLs individuales de cada evento
* Finalmente también haremos una lectura de tabla para finalmente recolectar los datos.

### Agradecimientos
En análisis se realizó entre **17 y 20 de Octubre** y específicamente el enfoque era encontrar puntos donde pudiera haber ilegalidad en la extracción de información de las fuentes de datos. Para ello, estuve mirando especialmente los ficheros de [robots.txt de Wikipedia](https://en.wikipedia.org/robots.txt), que dicho, es nuestra fuente de información, al cual dejamos el agradecimiento y sugiero donaciones para la iniciativa.
Las URLs de lectura no estaban *Disallowed* en el fichero de Robots, por lo tanto, no habría problemas explorarlas. Aun así, he aplicado técnicas como sleep recomienda **[4] Miller, C. (2017)** para evitar la sobrecarga (*throttling*) de la webpage. 

### Inspiración
Desde mi punto de vista, el conjunto de datos es interesante porque el ámbito de su aplicación es bastante amplio, para comprobar, voy a describir escenarios de su aplicación:

1. Primeramente, una aplicación podría ser para uso en periodismo deportivo - para enseñar datos y patrones del deporte.

2. Un deportista, que compete o no en el evento, podría sacar informaciones de métodos más comúnmente utilizados para encerrar una lucha y, de esa manera, prepararse para evitar que hicieran con él o mismo practicar las técnicas para intentar aplicarlas cuando esté competiendo.

3. La preparación física en un deporte es un punto clave. Entender donde suele pasar la mayor parte de fines de luchas puede también ayudar los profesionales de preparación física a entrenar sus atletas para que en cansancio no les quite la oportunidad de victoria.

4. Podríamos identificar patrones de acuerdo con el peso (categoría) de los atletas, es decir, puede que en una categoría más ligera estén acostumbrados a pelearse más de pie, mientras en otras categorías más pasadas suele ocurrir grappling (lucha de suelo). 

Seguramente hay otras numerosas posibilidades de aplicación.


### Licencia
Haciendo un breve estudio de las licencias presentadas, creo que la que se aplicaría más ampliamente a mi estudio sería **CC BY-NC-SA 4.0**, me explico: la licencia en cuestión permite:
* *Compartir*: copiar y redistribuir el material
* *Adaptar*: transformar y cambiar el material

Sin embargo, no permite el uso comercial del mismo - el que, siendo un trabajo de máster, tiene fines más académicos.

Abajo, una imagen de que trata la licencia elegida:
<img src="license.png" alt="License" style="width: 500px; height: 400px;"/>
<center>Figura 2: CC BY-NC-SA 4.0.</center> 

P.D.: Como el los repositorios de Github no había disponibilidad de utilizar la dicha licencia, por allí he definido el uso de **BSD 3-Clause** que básicamente define que las redistribuciones de generadas con base en el proyecto en cuestión deben ser hechas con notificación a priori. Ademas, garantiza que los nombres de los creadores del proyecto inicial no pueden ser usados para promover productos derivados del proyecto inicial.

### Código
En ese apartado, tendremos el código utilizado para la extracción de los datos y al final para la generación del CSV.

In [1]:
import sys
print(sys.version)

3.7.0 (v3.7.0:1bf9cc5093, Jun 26 2018, 23:26:24) 
[Clang 6.0 (clang-600.0.57)]


In [2]:
# Not necessary at all, but to demonstrate that I'm aware that BeautifulSoup4 must be installed
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install BeautifulSoup4

Requirement already up-to-date: pip in /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages (18.1)


In [3]:
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML
from time import sleep
import requests
import pandas as pd
import os

In [4]:
Events = []
base_url = 'https://en.wikipedia.org'
main_url = base_url + '/wiki/List_of_UFC_events'

In [5]:
def perform_http_get(url):
    """
    Note:
        This is a simple function that performs an http_get and if the result code is 200 proceed the return.  

    Args:
        url (str): A string.

    Returns:
        BeautifulSoup: a version parsed of the request content
    """
    
    
    r = requests.get(url)
    if r.status_code == 200:
        return BeautifulSoup(r.content, 'html.parser')
        

In [6]:
def extract_cell(cells, id_td):
    """
    Note:
        This method is aimed for returning the content of a given cell (td) decoded and without 
        additional blank spaces. 

    Args:
        cells (list): A list representing the table row.
        id_td (int): The index of the wanted cell

    Returns:
        String: the real content of the cell.
    """
    
    return cells[id_td].renderContents().decode().strip()

In [7]:
def append_fighter_names(cells_event, info):
    """
    Note:
        This method presents some logic that will be briefly explained here. It turns out that some
        fighters don't have their own wikipedia page, in these cases, there is no link (<a>) within
        their names. Therefore, returning just the content of the cell where this info is present 
        is enough. In other cases, where the link is there, we must extract to update our dictionary.
    Args:
        cells_event (list): A list representing the table row.
        info (dict): The dictionary to be updated
    """
    
    
    
    fighter1 = ''
    fighter2 = ''
    if len(cells_event[1].findAll('a')) == 0:
        fighter1 = extract_cell(cells_event, 1)
    else:
        fighter1 = cells_event[1].find('a').renderContents().decode().strip()

    if len(cells_event[3].findAll('a')) == 0:
        fighter2 = extract_cell(cells_event, 3)
    else:
        fighter2 = cells_event[3].find('a').renderContents().decode().strip()
    
    info.update({"fighter_1" : fighter1})
    info.update({"fighter_2" : fighter2})

In [8]:
def extract_row(cells_event, link):
    """
    Note:
        The goal here is to create a dictionary for the given events and also
        to update in our events list (Events.append). We realize that within
        the method we are also invoking the append_fighter_names that will call
        the aforementioned method to treat the fighters' name.

    Args:
        cells_event (list): A list representing the table row.
        link (str): Basically the event name
    """   
    info = {
        "event_name": link.contents[0],
        "weight_class": extract_cell(cells_event, 0),
        "action": extract_cell(cells_event, 2),
        "method": extract_cell(cells_event, 4),
        "round": extract_cell(cells_event, 5),
        "time": extract_cell(cells_event, 6)
    }
    
    append_fighter_names(cells_event, info)
    Events.append(info)

In [9]:
def download_event_image(individual_event, link):
    """
    Note:
        The method's name is pretty clear here, however, the objective is to 
        download the images of each event in order to present them at the bottom
        of this current document. In a nutshell, the behaviour expected here
        is to create all the recovered images in a folder called pictures
        which, by the way, we can observe that will be created.

    Args:
        individual_event (BeautifulSoup): A BeautifulSoup representing the page of a single event.
    """  
    
    event_images = individual_event.select('table.infobox a.image img[src]')
    if len(event_images) > 0:
        for img in event_images:
            img_url = 'http:' + img['src']
            r = requests.get(img_url)
            with open('pictures/' + link.contents[0], "wb") as code:
                code.write(r.content)
            

In [10]:
def extract_info_individual_event(link):
    """
    Note:
        As you will see, this is the second part of a for loop that I will summarize here:
        First we iterate over the list of UFC events presented in the main web-page (
        https://en.wikipedia.org/wiki/List_of_UFC_events). From the data gathered there
        each individual event will be accessed and we'll collect data from these events.
        
        Needless to say that I'm not iterating over a single web-page, but instead, more than
        450 different web pages, being the first of them the "master one", and this individual
        ones.

    Args:
        link (url): The URL for the individual web-pages of a given list.
    """  
    individual_event = perform_http_get(base_url + link.get('href'))
    table = individual_event.find('table',{'class': 'toccolours'})

    download_event_image(individual_event, link)
   
    if table is not None:
        rows_event = table.findAll('tr')

        for row_event in rows_event:
            cells_event = row_event.findAll('td')
            if len(cells_event) > 0 :
                extract_row(cells_event, link)
                        

In [11]:
"""
From my perspective this is a key part of the program and it will trigger 
the execution and the invocation of the previous methods.
Basically we are collecting the "Past Events" of a table in one page and
for each event in this table (currently 454), we will read another 
information as previously explained in different web pages.
By reading another information, I mean: download the event image, read
the card and the results of the event and so on.


Apart from the behaviour expected, in this part of the code, I also tried
to avoid throtling the Wikipedia source by adding some sleeps within each
iteration - it will slow down the process and avoid some common problems.
"""


Events = []
soup = perform_http_get(main_url)
table_past_events = soup.find('table', {'id': 'Past_events'})

rows = table_past_events.findAll('tr')

for row in rows:
    #sleep(10) # Wait 10 sec, recommendations explained by [4] Miller, C. (2017)
    cells = row.findAll('td')
    if len(cells) > 0 :
        links = cells[1].findAll('a')
        for link in links:        
            extract_info_individual_event(link)

In [12]:
"""
Once we have a list of dict in Events attributes, it's high time we defined
a panda dataframe to properly store the info.
It is done here and we present a glimpse of the data with the df.head() below.
"""



print(len(Events), ' eventos fueron añadidos')

df = pd.DataFrame(Events)
df = df[['event_name', 'weight_class', 'fighter_1', 'action', 'fighter_2', 'round', 'time', 'method' ]]

df.head(15)

4869  eventos fueron añadidos


,event_name,weight_class,fighter_1,action,fighter_2,round,time,method
0,UFC Fight Night: Volkan vs. Smith,Light Heavyweight,Anthony Smith,def.,Volkan Oezdemir,3,4:26,Submission (rear-naked choke)
1,UFC Fight Night: Volkan vs. Smith,Catchweight (147 lbs),Michael Johnson,def.,Artem Lobov,3,5:00,"Decision (unanimous) (29-28, 29-28, 30-27)"
2,UFC Fight Night: Volkan vs. Smith,Light Heavyweight,Misha Cirkunov,def.,Patrick Cummins,1,2:40,Submission (arm-triangle choke)
3,UFC Fight Night: Volkan vs. Smith,Bantamweight,Andre Soukhamthath,def.,Jonathan Martinez,3,5:00,"Decision (unanimous) (30-26, 29-28, 29-28)"
4,UFC Fight Night: Volkan vs. Smith,Light Heavyweight,Gian Villante,def.,Ed Herman,3,5:00,"Decision (split) (29-28, 28-29, 29-28)"
5,UFC Fight Night: Volkan vs. Smith,Welterweight,Court McGee,def.,Alex Garcia,3,5:00,"Decision (unanimous) (29-28, 29-28, 30-28)"
6,UFC Fight Night: Volkan vs. Smith,Welterweight,Sean Strickland,def.,Nordine Taleb,2,3:10,TKO (punches)
7,UFC Fight Night: Volkan vs. Smith,Lightweight,Nasrat Haqparast,def.,Thibault Gouti,3,5:00,"Decision (unanimous) (29-27, 29-28, 30-26)"
8,UFC Fight Night: Volkan vs. Smith,Featherweight,Calvin Kattar,def.,Chris Fishgold,1,4:11,TKO (punches)
9,UFC Fight Night: Volkan vs. Smith,Women's Bantamweight,Talita Bernardo,def.,Sarah Moras,3,5:00,"Decision (unanimous) (30-27, 29-28, 29-28)"


In [13]:
"""
This is particularly a visual part of the presentation of the data.
For each event, I'm presenting the folder advertising it.
Remembering that we download each of these pictures in a method 
explained before
"""


df_imgs = df['event_name'].drop_duplicates()

print('Imagenes: ', len(df_imgs))
display(HTML('<table style="width:100%; border: 1px solid black;"><tr>'))
i = 0
row = ''
for img in df_imgs:
    img_src = 'pictures/' + img
    if i == 20:
        row = '{}<td><img src=\'{}\' alt=\'{}\' /></td></tr>'.format(row, img_src, img)
        display(HTML(row))
        row = '<tr>'
        i = 0
    else:
        row = '{}<td><img src=\'{}\' /></td>'.format(row, img_src)
     
    i += 1

display(HTML('</tr></table>'))



Imagenes:  454


,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,


...

,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,


### Dataset: Dataset en formato CSV
Por aquí generamos el dataset que podrá ser consultado en el repositorio.

In [14]:
"""
Last but not least, we are storing the dataframe in a CSV file
"""

file_name = 'ufc-events.csv'
df.to_csv(file_name)

### Referencias

<p style="text-align: justify">
[1] <b>Subirats, L., Calvo, M. (2018).</b> "<i>Web Scraping</i>". Editorial UOC. Barcelona: Universitat Autònoma de Barcelona.<p>
<p style="text-align: justify">
[2] <b>Masip, D. (?).</b> "<i>El lenguaje Python</i>". Editorial UOC. Barcelona: Universitat Autònoma de Barcelona.<p>
<p style="text-align: justify">
[3] <b>Lawson, R. (2015).</b> "<i>Web Scraping with Python</i>". Packt Publishing Ltd. Chapter 2. Scraping the Data.<p>
<p style="text-align: justify">
[4] <b>Miller, C. (2017).</b> "<i>Data Acquisition and Manipulation with Python - Acquire, Analyse, and Play with Data</i>". Packt Publishing Ltd. Chapter 2. Web Scraping with BeautifulSoup.<p>
    
<p style="text-align: justify">
[5] <b>Chhibber, A. (2018).</b> "<i>Web Scraping Using Python</i>". Technics Publications.<p>